**Content based recommendation**

In [11]:
# Example books with genres/tags
books = {
    "The Hobbit": ["fantasy", "adventure", "classic"],
    "Harry Potter": ["fantasy", "magic", "adventure"],
    "1984": ["dystopia", "classic", "political"],
    "The Lord of the Rings": ["fantasy", "adventure", "epic"],
    "Brave New World": ["dystopia", "classic", "science fiction"],
    "The Catcher in the Rye": ["classic", "coming of age", "literature"],
    "Animal Farm": ["dystopia", "political", "classic"],
    "Percy Jackson": ["fantasy", "adventure", "mythology"]
}

# User likes books
user_likes = ["Harry Potter", "The Hobbit"]

In [12]:
# cosine similarity based on tags
def cosine_sim(a, b):
    a_set, b_set = set(a), set(b)
    return len(a_set & b_set) / (len(a_set) * len(b_set))**0.5

In [14]:
# Aggregate similarity (sum over liked books)
recommendations = {}
for book, tags in books.items():
    if book not in user_likes:
        sim = sum([cosine_sim(tags, books[liked]) for liked in user_likes])
        recommendations[book] = sim

# Sort and pick top 3
top_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:3]

print("Top 3 Content-Based Book Recommendations:")
for book, score in top_recommendations:
    print(f"{book} (score: {round(score,2)})")


Top 3 Content-Based Book Recommendations:
The Lord of the Rings (score: 1.33)
Percy Jackson (score: 1.33)
1984 (score: 0.33)


**Collaborative based recommendation**

In [20]:
# Users and ratings (1-5 scale)
ratings = {
    "User1": {"Inception":5, "Avatar":4},      # User1 has not rated Titanic
    "User2": {"Inception":4, "Avatar":5, "Titanic":2},
    "User3": {"Inception":2, "Avatar":1, "Titanic":5},
    "User4": {"Inception":5, "Avatar":4, "Titanic":3}
}

target_user = "User1"

In [21]:
# Pearson correlation
def pearson_sim(user1, user2):
    common = set(ratings[user1]) & set(ratings[user2])
    if not common: return 0
    x = [ratings[user1][m] for m in common]
    y = [ratings[user2][m] for m in common]
    n = len(common)
    num = sum((xi - sum(x)/n)*(yi - sum(y)/n) for xi, yi in zip(x,y))
    den = (sum((xi - sum(x)/n)**2 for xi in x) * sum((yi - sum(y)/n)**2 for yi in y))**0.5
    return num/den if den !=0 else 0

In [22]:
# Predict unrated movies
all_movies = set(m for r in ratings.values() for m in r)
predictions = {}
for movie in all_movies:
    if movie not in ratings[target_user]:   # Only predict for unseen movies
        score = 0
        sim_sum = 0
        for user, user_ratings in ratings.items():
            if user != target_user and movie in user_ratings:
                sim = pearson_sim(target_user, user)
                score += sim * user_ratings[movie]
                sim_sum += abs(sim)
        predictions[movie] = score/sim_sum if sim_sum!=0 else 0

# Sort predictions
predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
print("Collaborative Filtering Recommendations for User1:")
for movie, score in predictions:
    print(f"{movie} (predicted score: {round(score,2)})")

Collaborative Filtering Recommendations for User1:
Titanic (predicted score: 2.0)
